In [1]:
import pandas as pd
import os
from rdkit import Chem

pattern_triazolopyrazine = Chem.MolFromSmarts("*-c1nnc2cncc(-*)n12")
def is_triazolopyrazine(mol):
    if mol.HasSubstructMatch(pattern_triazolopyrazine):
        return 1
    else:
        return 0

DATAPATH = "../data"

In [2]:
#load already synthesized molecules and compare with mollib
gen_mols = pd.read_csv(os.path.join(DATAPATH, "data_0.csv"))
real_mols = pd.read_csv(os.path.join(DATAPATH, "training_all.csv"))
gen_smi = gen_mols["Smiles"].tolist()
real_smi = real_mols["smiles"].tolist()

In [11]:
#read mollib output
mollib_40e_dict = {
    "ep10": pd.read_csv(os.path.join(DATAPATH, "mollib_40epochs", "molecules_10_0.7.txt"), header = None),
    "ep20": pd.read_csv(os.path.join(DATAPATH, "mollib_40epochs", "molecules_20_0.7.txt"), header = None),
    "ep30": pd.read_csv(os.path.join(DATAPATH, "mollib_40epochs", "molecules_30_0.7.txt"), header = None),
    "ep40": pd.read_csv(os.path.join(DATAPATH, "mollib_40epochs", "molecules_40_0.7.txt"), header = None),
    }
for k,v in mollib_40e_dict.items():
    v.columns=["smiles"]
for k,v in mollib_40e_dict.items():
    print(len(v))

339
325
345
327


In [12]:
mollib_40e = pd.concat([v for k,v in mollib_40e_dict.items()], ignore_index=True)
smiles = mollib_40e["smiles"].tolist()
mols = [Chem.MolFromSmiles(smi) for smi in smiles]
mollib_40e["IsTriazoloPyrazine"] = [is_triazolopyrazine(mol) for mol in (mols)]
idx = mollib_40e.index[(mollib_40e["IsTriazoloPyrazine"]==0)].tolist()
mollib_40e.drop(idx, inplace=True)
print(mollib_40e.shape)

(1214, 2)


In [13]:
mollib_40e_smi = mollib_40e["smiles"].tolist()

dup_smi = list(set(gen_smi).intersection(mollib_40e_smi))
print(len(dup_smi))
mollib_new = list(set(mollib_40e_smi).difference(set(gen_smi)))
print(len(mollib_new))
red_smi = list(set(real_smi).intersection(set(mollib_40e_smi)))
print(len(red_smi))
mollib_40e_new = list(set(mollib_new).difference(set(real_smi)))
print(len(mollib_40e_new))
mollib_40e_new = list(set(mollib_40e_new).difference(set(mollib_10e_new)))
print(len(mollib_40e_new))

mollib_data = pd.DataFrame(mollib_40e_new, columns=["smiles"])
mollib_data.to_csv(os.path.join(DATAPATH, "mollib_40e.csv"), index=False)

274
759
46
713
683
